**Importing Libraries**

In [13]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from sklearn.model_selection  import train_test_split
import itertools
import math
!pip install wandb
import wandb
%matplotlib inline

In [14]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

**`Loading and processing data`**

In [15]:
(train_images, train_labels),(test_images, test_labels) = fashion_mnist.load_data()

In [16]:
sample_labels = list(train_labels)
names = ["T-shirt/top","Trouser/pants","Pullover shirt","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]
for i in range(10):
    wandb.init(project="Assignment1")
    image_index = list.index(sample_labels, i)
    print(image_index)
    wandb.log({'label': i, 'image': [wandb.Image(train_images[image_index], caption='{}'.format(names[i]))]})

label,9
_runtime,3
_timestamp,1615562511
_step,0


label,▁
_runtime,▁
_timestamp,▁
_step,▁


1


label,0
_runtime,3
_timestamp,1615563331
_step,0


label,▁
_runtime,▁
_timestamp,▁
_step,▁


...

label,▁
_runtime,▁
_timestamp,▁
_step,▁


23


label,8
_runtime,3
_timestamp,1615563384
_step,0


label,▁
_runtime,▁
_timestamp,▁
_step,▁


0


In [17]:
train_images, val_images, train_labels, val_labels  = train_test_split(train_images,train_labels,test_size=0.1,random_state = 42)

In [18]:
traind = train_images.reshape(train_images.shape[0],-1)
mean = traind.mean(axis=0)
centerd = traind -  mean
max = centerd.max(axis=0)
normalized = centerd/np.max(max)


val_images = val_images.reshape(val_images.shape[0],-1)
mean = val_images.mean(axis=0)
centerd = val_images -  mean
max = centerd.max(axis=0)
val_images = centerd/np.max(max)

In [19]:
#parameters:
Size_of_Input = normalized.shape[1]
Number_of_Neuron_each_Layer = [32,10]
Number_of_Layers = 2
activation_function = 'sigmoid'
weight_initializer = 'random'

In [32]:
class NeuralNet:
    def __init__(self,Size_of_Input, Number_of_Neuron_each_Layer, Number_of_Layers, activation_function, weight_initializer):
        self.activation_function = activation_function
        self.Size_of_Input = Size_of_Input
        self.Number_of_Layers = Number_of_Layers
        self.Number_of_Neuron_each_Layer = Number_of_Neuron_each_Layer
        self.W,self.b = self.initializer(weight_initializer)

    
    def initializer(self, weight_initializer):        
        W = []
        b = []
        if weight_initializer == 'random':
            W.append(np.random.randn(self.Number_of_Neuron_each_Layer[0], self.Size_of_Input))
            for i in range(1,self.Number_of_Layers):
                W.append(np.random.randn(self.Number_of_Neuron_each_Layer[i],self.Number_of_Neuron_each_Layer[i-1]))

            for i in range(self.Number_of_Layers):
                b.append(np.random.rand(self.Number_of_Neuron_each_Layer[i]))
        elif(weight_initializer == 'xavier'):
            W.append(np.random.normal(0,math.sqrt(2/(self.Number_of_Neuron_each_Layer[0]+ self.Size_of_Input)), (self.Number_of_Neuron_each_Layer[0], self.Size_of_Input)))
            for i in range(1,self.Number_of_Layers):
                W.append(np.random.normal(0, math.sqrt(2/(self.Number_of_Neuron_each_Layer[i]+self.Number_of_Neuron_each_Layer[i-1])),(self.Number_of_Neuron_each_Layer[i],self.Number_of_Neuron_each_Layer[i-1])))

            for i in range(self.Number_of_Layers):
                b.append(np.random.rand(self.Number_of_Neuron_each_Layer[i]))
            np.random.normal(0.0, 1.0, (2,2))
        return W,b


    def activation(self, Z):
        if self.activation_function == 'ReLU':
            return self.ReLU(Z)
        elif self.activation_function == 'tanh':
            return self.tanh(Z)
        elif self.activation_function == 'sigmoid':
            return self.sigmoid(Z)


    def activation_derivative(self,Z):
        if self.activation_function == 'ReLU':
            return self.ReLU_derivative(Z)
        elif self.activation_function == 'tanh':
            return self.tanh_derivative(Z)
        elif self.activation_function == 'sigmoid':
            return self.sigmoid_derivative(Z)

    def ReLU(self,Z):
        return np.maximum(0,Z)

    def ReLU_derivative(self,Z):
        return [1 if x>0 else 0 for x in Z]

    def tanh(self, Z):
        return np.array([((np.exp(x) - np.exp(-x))/((np.exp(x) + np.exp(-x)))) for x in Z])
                 
    def tanh_derivative(self, Z):
        return np.array(1 - self.tanh(Z)**2)
                 
    def sigmoid_derivative(self,Z):
        return self.sigmoid(Z)*(1-self.sigmoid(Z))

    def sigmoid(self,x):
        return np.where(x>=0, 1/(1+np.exp(-x)), np.exp(x)/(1+np.exp(x)))
    
    def softmax_function(self,Z):
        Z = Z - Z.max()
        return (np.exp(Z)/np.sum(np.exp(Z),axis=0))

    def forward_propogation(self,Input):
        A = []
        H = []
        Input = np.array(Input.T)
        A.append(self.W[0].dot(Input) + self.b[0].reshape(self.b[0].shape[0],1))
        for i in range(1, self.Number_of_Layers):
            H.append(self.activation(A[-1]))
            A.append(self.W[i].dot(H[-1]) + self.b[i].reshape(self.b[i].shape[0],1))
        y_hat = self.softmax_function(A[-1])
        return np.argmax(y_hat,axis=0)

In [33]:
model = NeuralNet(Size_of_Input, Number_of_Neuron_each_Layer, Number_of_Layers, activation_function, weight_initializer)
y_pred = model.forward_propogation(normalized)